In [1]:
#import libraries
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adam,RMSprop

Using TensorFlow backend.


In [2]:
path='D:/Datasets/Image dataset/eye disease/eye-disease-dataset/'


In [3]:
#size of images to feed in neural network
image_shape = (100,100,3) 
#data augmentation

datagen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               vertical_flip=True,
                               fill_mode='nearest',# Fill in missing pixels with the nearest filled value
                               validation_split=0.2#split data to train and test
                              )

In [4]:
#load the training data
train_generator = datagen.flow_from_directory(
    path+'Cropped_Images/',
    target_size=image_shape[0:2],
    batch_size=30,
    class_mode='categorical',
    subset='training')

Found 726 images belonging to 5 classes.


In [5]:
#load the test data
test_generator = datagen.flow_from_directory(
    path+'Cropped_Images/',
    target_size=image_shape[0:2],
    batch_size=30,
    class_mode='categorical',
    subset='validation')


Found 180 images belonging to 5 classes.


In [6]:
from keras.applications.vgg19 import VGG19
model =VGG19(weights = "imagenet", include_top=False, input_shape = image_shape)
#model.layers

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [8]:
for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 
x = model.output
#x=Conv2D(filters = 64,kernel_size = 5,activation='relu',padding='same')(x)
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

In [9]:
model = Model(inputs=model.input, outputs=predictions)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [11]:
#compile the model
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
#create class weights for unbalance dataset
from sklearn.utils import class_weight
y_train=train_generator.classes
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))


In [13]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=5)

In [14]:
#train model
results = model.fit_generator(train_generator,epochs=40,
                              steps_per_epoch=30,
                              validation_data=test_generator,
                             validation_steps=30,
                              callbacks=[es],
                             class_weight=class_weights)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/40
30/30 [==============================] - 7s 217ms/step - loss: 1.6202 - acc: 0.2477 - val_loss: 1.3952 - val_acc: 0.5467
Epoch 2/40
30/30 [==============================] - 4s 133ms/step - loss: 1.4408 - acc: 0.3944 - val_loss: 1.3800 - val_acc: 0.4056
Epoch 3/40
30/30 [==============================] - 4s 135ms/step - loss: 1.3425 - acc: 0.4358 - val_loss: 1.2678 - val_acc: 0.4544
Epoch 4/40
30/30 [==============================] - 4s 135ms/step - loss: 1.3476 - acc: 0.4377 - val_loss: 1.4397 - val_acc: 0.4122
Epoch 5/40
30/30 [==============================] - 4s 134ms/step - loss: 1.3230 - acc: 0.4477 - val_loss: 1.3331 - val_acc: 0.4378
Epoch 6/40
30/30 [==============================] - 4s 133ms/step - loss: 1.2503 - acc: 0.4801 - val_loss: 1.3475 - val_acc: 0.4300
Epoch 7/40
30/30 [==============================] - 4s 133ms/step - loss: 1.2035 - acc: 0.5235 - val_loss: 1.2447 

In [15]:
#print classification report
y_true=test_generator.classes
class_label=list(train_generator.class_indices.keys())
from sklearn.metrics import classification_report
y_pred = model.predict_generator(test_generator,steps=180/30) #186 are total example in test set and 40 is batch size
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_true, y_pred,target_names=class_label))


              precision    recall  f1-score   support

Bulging_Eyes       0.30      0.33      0.31        43
   Cataracts       0.15      0.18      0.17        11
Crossed_Eyes       0.54      0.35      0.43        88
    Glaucoma       0.18      0.14      0.15        22
     Uveitis       0.11      0.31      0.16        16

    accuracy                           0.31       180
   macro avg       0.26      0.26      0.24       180
weighted avg       0.38      0.31      0.33       180



In [16]:
K.clear_session()